# Observer: property dependencies

> Exposing the problems with property observers

**Property observers** seem very useful but unfortunately there are problems with them, one of them being what happens when a property is dependent on another property.

We will reuse the code from the last lesson but we will now add a new boolean property that will indicate whether a `Person` can vote.

In [1]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

class PropertyObservable:
    def __init__(self):
        self.property_changed = Event()

class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if self._age == value:
            return
        self._age = value
        self.property_changed('age', value)

    # new boolean property
    @property
    def can_vote(self):
        return self.age >= 18

Our new `can_vote` property is a computed property, so there is no setter to worry about. However, with our current code, it's simply impossible to send notifications on changes to the voting ability of a `Person`. Let's try it anyway:

In [2]:
# subscriber
def person_changed(name, value):
    if name == 'can_vote':
        print(f'Voting status changed to {value}')

In [3]:
p = Person()
p.property_changed.append(person_changed)

for age in range(16, 21):
    print(f'Changing age to {age}')
    p.age = age

Changing age to 16
Changing age to 17
Changing age to 18
Changing age to 19
Changing age to 20


As you can see in the code above, even though `person_changed` checks for the `can_vote` property, it's never triggered because we never call `property_changed` with `can_vote`.

You might be tempted to trigger a second event within the `age` setter for `can_vote`, but that would be incorrect because the property change only happens when the `Person` turns 18, thus it doesn't make sense to trigger an event for every age change.

What we will do is caching the old `can_vote` value in the `age` setter before actually changing the age, then do a comparison between the cached value and the current value, and then trigger the event is it has changed:

In [4]:
class Person(PropertyObservable):
    def __init__(self, age=0):
        super().__init__()
        self._age = age

    @property
    def age(self):
        return self._age
    
    @property
    def can_vote(self):
        return self.age >= 18

    @age.setter
    def age(self, value):
        if self._age == value:
            return
        
        # we cache the old can_vote value
        cached_can_vote = self.can_vote

        # we change the age as usual
        self._age = value
        self.property_changed('age', value)

        # we check whether can_vote changed and trigger the second event if true
        if cached_can_vote != self.can_vote:
            self.property_changed('can_vote', self.can_vote)

We are now ready to try again:

In [5]:
p = Person()
p.property_changed.append(person_changed)

for age in range(16, 21):
    print(f'Changing age to {age}')
    p.age = age

Changing age to 16
Changing age to 17
Changing age to 18
Voting status changed to True
Changing age to 19
Changing age to 20


The code now works as expected.

However, note that the code is more complex than in the previous lesson. Property observers don't scale well and in more complex scenarios with multi-level dependencies, the code gets messy fast.